# Práctica: Word2Vec / Skip-gram
En este trabajo he implementado desde cero un modelo Word2Vec basado en Skip-Gram con el objetivo de entender su funcionamiento interno. Se ha trabajado con un corpus controlado, diseñado para contener patrones lingüísticos claros como relaciones de género gramatical, asociaciones entre agentes y acciones, y relaciones geográficas simples.

Se ha utilizado softmax completo sobre todo el vocabulario, siendo consciente de su coste computacional, porque permite seguir de forma directa la función objetivo original y analizar con claridad el proceso de entrenamiento. Alternativas más eficientes como Negative Sampling no se han incluido para priorizar la comprensión conceptual del modelo.

La evaluación combina un análisis cuantitativo, mediante la pérdida final y la similitud media entre palabras y sus vecinos más cercanos, con un análisis cualitativo basado en vecinos y analogías no triviales, diseñadas en función del contenido del corpus. Los resultados muestran que el modelo captura regularidades distribucionales básicas, aunque también evidencian las limitaciones derivadas del tamaño del corpus.
Se ejecuta el proyecto en el siguiente orden:
1. Carga y tokenización
2. Vocabulario
3. Pares Skip-gram (centro, contexto)
4. Entrenamiento Skip-gram
5. Evaluación: vecinos + analogías
6. Comentario sobre hiperparámetros

## Importación de librerías 

In [1]:
import os
import sys
import pandas as pd

PROJECT_ROOT = os.getcwd()
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from word2vec.data import load_and_tokenize
from word2vec.vocab import build_vocabulary
from word2vec.pairs import generate_skipgram_pairs
from word2vec.model import SkipGram
from word2vec.analysis import analogy_test, average_neighbor_similarity

## Configuración de hiperparámetros



In [2]:
CORPUS_PATH = "resources/dataset_word2vec.txt"

WINDOW_SIZE = 2
EMBEDDING_DIM = 30
LEARNING_RATE = 0.01
EPOCHS = 50
MIN_COUNT = 1

TOP_K_NEIGHBORS = 8

## Ejecución principal del programa

In [3]:
# 1) Cargar y tokenizar
sentences, all_words = load_and_tokenize(
    CORPUS_PATH,
    strip_accents=False,
    keep_numbers=False
)

# 2) Vocabulario
vocab, inv_vocab, counter = build_vocabulary(all_words, min_count=MIN_COUNT)

print("Tamaño del vocabulario:", len(vocab))
print("Top 10 palabras:", counter.most_common(10))


Tamaño del vocabulario: 355
Top 10 palabras: [('la', 131), ('el', 129), ('es', 37), ('está', 27), ('en', 26), ('un', 20), ('por', 15), ('una', 15), ('al', 14), ('gato', 11)]


## Búsqueda de hiperparámetros
En esta sección se entrenan varios modelos con combinaciones de hiperparámetros y se selecciona el que obtiene menor pérdida en la última época.


In [4]:
test_words = ["perro", "gata", "coche", "parís", "francia", "casa", "agua", "niña", "profesor", "médico"]
test_indices = [vocab[w] for w in test_words if w in vocab]

def score_analogies(model):
    analogies = [
        ("parís", "francia", "madrid"),
        ("perro", "ladra", "gato"),
        ("niño", "niña", "profesor"),
    ]
    score = 0.0
    used = 0
    for a, b, c in analogies:
        res = analogy_test(model, vocab, inv_vocab, a, b, c, top_k=1)
        if res is None:
            continue
        score += res[0][1]
        used += 1
    return score / used if used > 0 else 0.0


grid = {
    "window_size": [2, 3],
    "embedding_dim": [20, 30, 50],
    "learning_rate": [0.005, 0.01],
    "epochs": [30, 50, 100],
}

results = []

for ws in grid["window_size"]:
    pairs = generate_skipgram_pairs(sentences, vocab, window_size=ws)

    for dim in grid["embedding_dim"]:
        for lr in grid["learning_rate"]:
            for ep in grid["epochs"]:
                model_tmp = SkipGram(len(vocab), embedding_dim=dim, learning_rate=lr)

                # IMPORTANTÍSIMO: esto solo será correcto si ya corregiste el bug del return en model.train()
                final_loss = model_tmp.train(pairs, epochs=ep, verbose_every=0)

                neigh_score = average_neighbor_similarity(model_tmp, test_indices, top_k=5) if test_indices else 0.0
                anal_score = score_analogies(model_tmp)

                results.append({
                    "window_size": ws,
                    "embedding_dim": dim,
                    "learning_rate": lr,
                    "epochs": ep,
                    "final_loss": final_loss,
                    "avg_neighbor_sim": neigh_score,
                    "avg_analogy_top1_sim": anal_score,
                })

df_grid = pd.DataFrame(results).sort_values("final_loss", ascending=True)
df_grid.head(10)


,window_size,embedding_dim,learning_rate,epochs,final_loss,avg_neighbor_sim,avg_analogy_top1_sim
17,2,50,0.010,100,3.200688,0.779667,0.890657
11,2,30,0.010,100,3.277018,0.796621,0.920835
35,3,50,0.010,100,3.312844,0.793965,0.853818
5,2,20,0.010,100,3.337183,0.813248,0.921360
29,3,30,0.010,100,3.373771,0.811830,0.889943
23,3,20,0.010,100,3.419776,0.826954,0.888906
14,2,50,0.005,100,3.951078,0.880219,0.963678
16,2,50,0.010,50,3.995230,0.882428,0.963829
8,2,30,0.005,100,4.020109,0.898343,0.971051
10,2,30,0.010,50,4.061947,0.901169,0.972566


## Mejor resultado de hiperparámetros

In [5]:
best = df_grid.iloc[0]
print("Mejor combinación encontrada (menor pérdida final):")
print(best)

best_ws  = int(best["window_size"])
best_dim = int(best["embedding_dim"])
best_lr  = float(best["learning_rate"])
best_ep  = int(best["epochs"])

Mejor combinación encontrada (menor pérdida final):
window_size               2.000000
embedding_dim            50.000000
learning_rate             0.010000
epochs                  100.000000
final_loss                3.200688
avg_neighbor_sim          0.779667
avg_analogy_top1_sim      0.890657
Name: 17, dtype: float64


## Reentrenar el modelo con los mejores hiperparámetros
Se reentrena un modelo final con la mejor configuración para obtener las mejores analogías y vecinos.

In [6]:
best_pairs = generate_skipgram_pairs(sentences, vocab, window_size=best_ws)

best_model = SkipGram(len(vocab), embedding_dim=best_dim, learning_rate=best_lr)
best_final_loss = best_model.train(best_pairs, epochs=best_ep, verbose_every=10)

print("\nPérdida final del modelo seleccionado:", best_final_loss)


Epoch 10/100: Loss: 5.5937
Epoch 20/100: Loss: 4.7716
Epoch 30/100: Loss: 4.4778
Epoch 40/100: Loss: 4.2305
Epoch 50/100: Loss: 3.9947
Epoch 60/100: Loss: 3.7803
Epoch 70/100: Loss: 3.6031
Epoch 80/100: Loss: 3.4520
Epoch 90/100: Loss: 3.3172
Epoch 100/100: Loss: 3.2016

Pérdida final del modelo seleccionado: 3.201640312171827


## Vecinos más cercanos

In [7]:
def nearest_neighbors(words, top_k=8):
    rows = []
    for w in words:
        if w not in vocab:
            rows.append({"Palabra": w, f"Vecinos (top-{top_k})": "(no está en el vocabulario)"})
            continue
        idx = vocab[w]
        nn = best_model.nearest_neighbors(idx, top_k=top_k)
        neigh_words = [inv_vocab[j] for j, _ in nn]
        rows.append({"Palabra": w, f"Vecinos (top-{top_k})": ", ".join(neigh_words)})
    return pd.DataFrame(rows)

target_words = ["perro", "gata", "coche", "parís", "francia"]
nearest_neighbors(target_words, top_k=TOP_K_NEIGHBORS)


,Palabra,Vecinos (top-8)
0,perro,"gato, ratón, pequeño, perra, verdura, gata, la..."
1,gata,"perra, cuerda, cacao, alumna, hermana, bebo, v..."
2,coche,"lento, rápido, cuadro, verano, pasa, espera, c..."
3,parís,"madrid, roma, españa, francia, italia, ciudad,..."
4,francia,"italia, españa, palacio, madrid, parís, roma, ..."


## Analogías

In [8]:
analogies = [
    ("parís", "francia", "madrid"),
    ("perro", "ladra", "gato"),
    ("niño", "niña", "profesor"),
]

for a, b, c in analogies:
    res = analogy_test(best_model, vocab, inv_vocab, a, b, c, top_k=3)
    print(f"\n{a} : {b} :: {c} : ?")
    if res is None:
        print("  (alguna palabra no está en el vocabulario)")
    else:
        for w, sim in res:
            print(f"  - {w}: {sim:.4f}")



parís : francia :: madrid : ?
  - españa: 0.9985
  - italia: 0.9985
  - palacio: 0.8780

perro : ladra :: gato : ?
  - maúlla: 0.7821
  - pequeño: 0.7149
  - persigue: 0.7049

niño : niña :: profesor : ?
  - profesora: 0.9054
  - bicicleta: 0.8365
  - moto: 0.8229


# Conclusiones

### Análisis general
Los resultados que se han obtenido muestran que el modelo Skip-gram es capaz de aprender relaciones básicas entre palabras a partir del corpus proporcionado, que es bastante sencillo y no muy grande. En general, los vecinos más cercanos de una palabra suelen estar relacionados con su significado o con su uso habitual, como ocurre en ejemplos del tipo perro–gato o parís–francia–madrid-españa. Esto indica que los embeddings aprendidos capturan información relevante del contexto en el que aparecen las palabras.

### Análisis de analogías
Las analogías permiten observar de forma clara este comportamiento. Más concretamente, el modelo es capaz de resolver correctamente analogías sencillas como parís : francia :: madrid : españa, así como algunas transformaciones relacionadas con el género gramatical, por ejemplo niño : niña :: profesor : profesora. Aunque no todas las analogías funcionan perfectamente, los resultados obtenidos son razonables teniendo en cuenta que el corpus no es muy grande.

### Análisis de hiperparámetros
El análisis de hiperparámetros muestra que la elección de los mismos juegan un papel importante en el desarrollo del programa. En este caso, la mejor configuración corresponde a un tamaño de ventana de contexto intermedio, una dimensión de embedding relativamente grande y un número elevado de épocas de entrenamiento. Un tamaño de ventana de 2 permite capturar suficiente información del contexto sin introducir demasiado ruido, mientras que una dimensión de embedding mayor proporciona al modelo más capacidad para representar relaciones entre palabras. Por su parte, entrenar durante más épocas permite que el modelo ajuste mejor los vectores y reduzca la pérdida final, siempre que el corpus no sea excesivamente grande.

### Conclusión general
Por otro lado, el trabajo también nos muestra que existen algunas limitaciones. El corpus que se utiliza es pequeño, lo que restringe la complejidad de las relaciones que el modelo puede aprender, y el uso de softmax completo hace que el entrenamiento sea más lento y poco escalable. Estas limitaciones son esperables en el contexto de la práctica y no invalidan los resultados obtenidos.
